# Solving TSP using pygenetic

In this example we are going to walk through the usage of GAEngine to solve the TSP problem 
The objective would be to find shortest path distance covering all places
<b>Each permutation of places represents a potential candidate solution for the problem</b>


## 1. Chromosome Representation

Each chromosome is encoded as a permutation of the places

This can be easily achieved by using the `RangeFactory` of `pygenetic`. <br/>
The `RangeFactory` takes the following parameters
* minValue = minimum value a gene can take = 0 <br/>
* maxValue = minimum value a gene can take = 7 <br/>
* duplicates = if duplicates are allowed = False <br/>
* noOfGenes = number of genes in the chromosome = 8

In [49]:
from pygenetic import ChromosomeFactory

factory = ChromosomeFactory.ChromosomeRangeFactory(noOfGenes=8,minValue=0,maxValue=7)

You can also check if the factory works as expected by calling `createChromosome` function and observing the chromosome produced by the factory

In [50]:
# Code to test if factory works as expected
for i in range(5):
    print('Chromosome created: ', factory.createChromosome())

Chromosome created:  [4, 7, 1, 2, 0, 3, 5, 6]
Chromosome created:  [3, 6, 0, 7, 5, 1, 2, 4]
Chromosome created:  [7, 6, 0, 5, 2, 1, 3, 4]
Chromosome created:  [7, 6, 1, 2, 5, 3, 0, 4]
Chromosome created:  [7, 6, 1, 2, 0, 5, 4, 3]


### Sample input matrix for TSP problem

In [51]:
matrix = [[0,172,145,607,329,72,312,120],
          [172,0,192,494,209,158,216,92],
          [145,192,0,490,237,75,205,100],
          [607,494,490,0,286,545,296,489],
          [329,209,237,286,0,421,49,208],
          [72,158,75,545,421,0,249,75],
          [312,216,205,296,49,249,9,194],
          [120,92,100,489,208,75,194,0]]

## 2. Fitness function 
Fitness for a given chromosome is to minimize the distance travelled while covering all places. Hence, we have a minimization GA problem.


In [52]:
def fitness(chromosome, matrix):
		total = 0
		for i in range(len(chromosome)-1):
			total += matrix[chromosome[i]][chromosome[i+1]]
		return total


We need then create a `GAEngine` instance from the `pygenetic` package and set the following
* `factory` = the range factory instance we had intially created
* `population_size = 500` would be a good number for this problem
* `cross_prob = 0.8`
* `mut_prob = 0.2`
* `fitness_type = ('min', 10)` since our objective in this GA is to achieve the fitness value of 10

In [55]:
from pygenetic import GAEngine, Utils
ga = GAEngine.GAEngine(factory,10,fitness_type='min',mut_prob = 0.02)

We can now add the fitness function we had defined to this `GAEngine` instance

In [56]:
ga.setFitnessHandler(fitness, matrix)

## 3. Determing other attributes of the GA

Many Standard Crossover, Mutation, Selection and Fitness functions are present in the `Utils` module of the `pygenetic` package.

In [57]:
from pygenetic import Utils

### Crossover
Traditional crossover methods such as 1-point, 2-point crossover cannot be used since it create duplicate genes in the offsprings. In the popularly used `distinct` crossover, the first half of the chromosome is kept the same while the second half is obtained by sequentially traversing the second chromosome and adding elements only if that element is not already present.
<img src="tsp-crossover.png" style="width:700px;">

This can be done using the `addCrossoverHandler` of the pygenetic module which takes as parameters
* crossover_function = the crossover function to be used
* weight = the weightage the crossover function needs to be given (mainly used when multiple crossovers are added)

In [58]:
ga.addCrossoverHandler(Utils.CrossoverHandlers.PMX, 9)
ga.addCrossoverHandler(Utils.CrossoverHandlers.distinct, 4)
ga.addCrossoverHandler(Utils.CrossoverHandlers.OX, 3)

### Mutation

The use of the mutation technique of `swap` as shown in the diagram also ensures that each element in the chromosome is a unique number and that there are no duplicates. This is a suitable for mutation function for this problem
<img src="tsp-mutation.png" style="width:700px">

This can be done using the `addMutationHandler` of the pygenetic module which takes as parameters
* mutation_function = the mutation function to be used
* weight = the weightage the mutation function needs to be given (mainly used when multiple mutations are added)

In [59]:
ga.addMutationHandler(Utils.MutationHandlers.swap)

## Selection
The selection function `best` chooses the best (1 - cross_prob) percent of the population. Hence, this function is one of the possible selection handlers which can be used in our genetic algorithm

In [60]:
ga.setSelectionHandler(Utils.SelectionHandlers.SUS)

## 4. Time to Evolve

This can be easily done using the `evolve` function of the GAEngine instance. It takes the `noOfIterations` as a parameter. Let's evolve it for 100 generations

In [61]:
ga.evolve(100)

mutation_handlers_weights =  [1.0]
crossover_handlers_weights =  [0.5625, 0.8125, 1.0]
Diversity =  93.42892485734811
New mutation value =  0.0373295128263303
Members left after selection =  10
Best member after selection =  [3, 7, 2, 5, 0, 6, 4, 1]
Best fitness after selection =  1306
crossover_indices =  [3 4 1 0 6 2]
adaptive_mutation value passed =  True
mutation_indexes =  []
New generation members =  [[3, 7, 2, 5, 0, 6, 4, 1], [3, 7, 2, 5, 0, 6, 4, 1], [5, 2, 0, 6, 4, 1, 7, 3], [5, 4, 7, 1, 3, 0, 2, 6], [5, 3, 0, 6, 2, 7, 4, 1], [5, 6, 2, 0, 4, 1, 7, 3], [3, 7, 5, 6, 4, 2, 0, 1], [1, 3, 2, 5, 0, 7, 6, 4], [1, 6, 7, 5, 2, 4, 3, 0], [2, 4, 3, 0, 5, 1, 6, 7]]
Length of new generation  10
Diversity =  98.66635191391237
New mutation value =  0.03719032632113738
Members left after selection =  10
Best member after selection =  [3, 7, 2, 5, 0, 6, 4, 1]
Best fitness after selection =  1306
crossover_indices =  [1 0 6 4 5 3]
adaptive_mutation value passed =  True
mutation_indexes =  []
Ne

We can get the best member by using the `best_fitness` attribute of the `GAEngine`. 
It returns a tuple having
* chromsome having best fitness
* best fitness value

In [62]:
best = ga.best_fitness
print(best)

([3, 6, 2, 5, 0, 7, 4, 1], 1185)


## 5. Plotting the Statistics

- The functionality for plotting the best, worst, average fitness values across iterations is present in `plot_statistics` function of statistics.py module. The function takes a list of attributes to be plotted.
- These attributes can be `best-fitness`,`worst-fitness`,`avg-fitness`, `'diversity`, `mutation_rate`
- The diversity and mutation rate values over iterations can also be visualized 

In [65]:
import matplotlib.pyplot as plt
fig = ga.statistics.plot_statistics(['best-fitness','worst-fitness','avg-fitness'])
plt.show()
fig = ga.statistics.plot_statistics(['diversity'])
plt.show()
fig = ga.statistics.plot_statistics(['mutation_rate'])
plt.show()

best-fitness [1306, 1306, 1306, 1306, 1306, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185]
worst-fitness [2172, 2186, 2186, 2204, 2186, 1835, 1306, 1306, 1306, 1306, 1306, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185, 1185]
avg-fitness [1748.6, 1642.9, 1648.2, 1508.8, 1414.2, 1346.8, 1281.8, 1245.5, 1233.4, 1221.3, 1197.1, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0, 1185.0]
diversity [93.42892485734811, 98.66635191391237, 126.26724040700344, 110.11964402412495, 83.55426978916158, 52.71011288168525, 15.30542387521496, 19.131779844018695, 18.7452393956439, 17.53456586288922, 11.479067906411217, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mutation_rate [0.0373295128263303, 0.03719032632113738, 0.036473640202197954, 0.03688953840902305, 0.03759478930457781, 0.038296528004963505, 0.03948994902228131, 0.